In [5]:
import pandas as pd
import tensorflow as tf
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense

In [3]:
## Someone else is using the GPU, so I am using the CPU here.

num_cores = 8

CPU, GPU = True, False
if GPU:
    num_GPU = 1
    num_CPU = 1
if CPU:
    num_CPU = 1
    num_GPU = 0

config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,\
        inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
        device_count = {'CPU' : num_CPU, 'GPU' : num_GPU})
session = tf.Session(config=config)
K.set_session(session)

In [6]:
# read in the file
data = pd.read_csv('CASP.csv')

In [7]:
def baseline_model():
    model = Sequential()
    model.add(Dense(units=1, activation='linear', input_dim=9))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [8]:
X = data.loc[:, 'F1':'F9'].as_matrix()
y = data.loc[:,'RMSD'].as_matrix()

In [ ]:
% time
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, StratifiedKFold

#hyperparameters to search through with CV
# helpful thing here? https://github.com/keras-team/keras/issues/4278
# https://machinelearningmastery.com/use-keras-deep-learning-models-scikit-learn-python/

parameters = {
    'epochs' : [200, 400],
    'batch_size' : [256,512], # The larger the batch size, the faster the model will run.
        'verbose' : [0] # for debugging purpose. It will significantly lower the running speed. To speed up, change it to 0.
}

#searching through all combinations of parameters to find best (ranked by CV)
reg = GridSearchCV(KerasRegressor(build_fn=baseline_model), parameters, cv = 5, scoring = 'neg_mean_squared_error', return_train_score = True)
reg.fit(X, y)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 15.3 µs


In [14]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=5, shuffle=True)
results = cross_val_score(reg, X, y, cv=kfold)

In [15]:
print (results)

[ -1828.95561056  -2406.42887063   -523.58724931  -1773.93858087
 -23802.79237422]


In [11]:
reg.cv_results_

{'mean_fit_time': array([20.3740963 , 41.11900005, 12.33722167, 25.06952462]),
 'mean_score_time': array([0.02472973, 0.04610882, 0.06301317, 0.09294605]),
 'mean_test_score': array([ -4534.65174027,   -200.72765929, -20068.10505265,  -2147.47085473]),
 'mean_train_score': array([ -4538.62839445,   -199.4333615 , -21738.0645604 ,  -2187.69938223]),
 'param_batch_size': masked_array(data=[256, 256, 512, 512],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_epochs': masked_array(data=[200, 400, 200, 400],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_verbose': masked_array(data=[0, 0, 0, 0],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'batch_size': 256, 'epochs': 200, 'verbose': 0},
  {'batch_size': 256, 'epochs': 400, 'verbose': 0},
  {'batch_size': 512, 'epochs': 200, 'verbose': 0},
  {'batch_si

In [ ]:
# https://stackoverflow.com/questions/40690598/can-keras-with-tensorflow-backend-be-forced-to-use-cpu-or-gpu-at-will